In [1]:
import geopandas as gpd
import pandas as pd
import fiona
import matplotlib.pyplot as plt
from shapely.geometry import Point
from scipy.spatial import cKDTree
from tqdm import tqdm
import rasterio
import numpy as np
import yaml
import os

# Load configuration
with open('../../config.yaml', 'r') as file:
    config = yaml.safe_load(file)

def resolve_path(relative_path):
    return os.path.join(config['base_path'], relative_path)

In [3]:
# Load the groundwater productivity data
df_GW = pd.read_csv(resolve_path(config['Groundwater_Productivity_path']), sep=r'\s+')

# Create geometry for groundwater productivity points
geometry = [Point(xy) for xy in tqdm(zip(df_GW['X'], df_GW['Y']), desc="Creating geometries")]
gdf_GW = gpd.GeoDataFrame(df_GW, geometry=geometry)
gdf_GW.set_crs(epsg=4326, inplace=True)
gdf_GW.to_file(resolve_path(config['Groundwater_Prod_gpkg_path']))
gdf_GW = gdf_GW.to_crs(epsg=3857)


# Map groundwater productivity to categories
value_map_cat = {'VH': 20.0, 'H': 12.5, 'M': 3.0, 'LM': 0.75, 'L': 0.3, 'VL': 0.1}

reverse_map_cat = {v: k for k, v in value_map_cat.items()}

gdf_GW['GW_prod_cat_L_s'] = gdf_GW['GWPROD_V2'].map(value_map_cat)



Creating geometries: 1019265it [00:07, 144288.70it/s]


In [4]:
# Load the groundwater productivity data
df_GW = pd.read_csv(resolve_path(config['Groundwater_Productivity_path']), sep=r'\s+')

# Create geometry for groundwater productivity points
geometry = [Point(xy) for xy in tqdm(zip(df_GW['X'], df_GW['Y']), desc="Creating geometries")]
gdf_GW = gpd.GeoDataFrame(df_GW, geometry=geometry)
gdf_GW.set_crs(epsg=4326, inplace=True)

# Map groundwater productivity to categories
value_map_cat = {'VH': 20.0, 'H': 12.5, 'M': 3.0, 'LM': 0.75, 'L': 0.3, 'VL': 0.1}
reverse_map_cat = {v: k for k, v in value_map_cat.items()}

gdf_GW['GW_prod_cat_L_s'] = gdf_GW['GWPROD_V2'].map(value_map_cat)
gdf_GW.to_file(resolve_path(config['Groundwater_Prod_gpkg_path']))

gdf_GW = gdf_GW.to_crs(epsg=3857)



Creating geometries: 1019265it [00:06, 162497.01it/s]


In [3]:
print(gdf_GW.head())

         X          Y GWPROD_V2                         geometry  \
0  9.41405  37.251192         M  POINT (1047967.285 4474177.758)   
1  9.46405  37.251192         M  POINT (1053533.259 4474177.758)   
2  9.51405  37.251192         M  POINT (1059099.234 4474177.758)   
3  9.56405  37.251192         M  POINT (1064665.209 4474177.758)   
4  9.61405  37.251192         H  POINT (1070231.183 4474177.758)   

   GW_prod_cat_L_s  
0              3.0  
1              3.0  
2              3.0  
3              3.0  
4             12.5  
